# Batch Recommendations using HRNN and User-Item-Interaction Data <a class="anchor" id="top"></a>

In this notebook, you will choose a dataset and prepare it for use with Amazon Personalize Batch Recommendations.

1. [Choose a dataset or data source](#source)
1. [Prepare your data](#prepare)
1. [Create dataset groups and the interactions dataset](#group_dataset)
1. [Configure an S3 bucket and an IAM role](#bucket_role)
1. [Import the interactions data](#import)
1. [Create solutions](#solutions)
1. [Batch recommendations](#batch)
1. [Clean up](#cleanup)

## Introduction <a class="anchor" id="intro"></a>

For the most part, the algorithms in Amazon Personalize (called recipes) look to solve different tasks, explained here:

1. **HRNN & HRNN-Metadata** - Recommends items based on previous user interactions with items.
1. **HRNN-Coldstart** - Recommends new items for which interaction data is not yet available.
1. **Personalized-Ranking** - Takes a collection of items and then orders them in probable order of interest using an HRNN-like approach.
1. **SIMS (Similar Items)** - Given one item, recommends other items also interacted with by users.
1. **Popularity-Count** - Recommends the most popular items, if HRNN or HRNN-Metadata do not have an answer - this is returned by default.

No matter the use case, the algorithms all share a base of learning on user-item-interaction data which is defined by 3 core attributes:

1. **UserID** - The user who interacted
1. **ItemID** - The item the user interacted with
1. **Timestamp** - The time at which the interaction occurred

We also support event types and event values defined by:

1. **Event Type** - Categorical label of an event (browse, purchased, rated, etc).
1. **Event Value** - A value corresponding to the event type that occurred. Generally speaking, we look for normalized values between 0 and 1 over the event types. For example, if there are three phases to complete a transaction (clicked, added-to-cart, and purchased), then there would be an event_value for each phase as 0.33, 0.66, and 1.0 respectfully.

The event type and event value fields are additional data which can be used to filter the data sent for training the personalization model. In this particular exercise we will not have an event type or event value. 

## Choose a dataset or data source <a class="anchor" id="source"></a>
[Back to top](#top)

As we mentioned, the user-item-iteraction data is key for getting started with the service. This means we need to look for use cases that generate that kind of data, a few common examples are:

1. Video-on-demand applications
1. E-commerce platforms
1. Social media aggregators / platforms

There are a few guidelines for scoping a problem suitable for Personalize. We recommend the values below as a starting point, although the [official limits](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) lie a little lower.

* Authenticated users
* At least 50 unique users
* At least 100 unique items
* At least 2 dozen interactions for each user 

Most of the time this is easily attainable, and if you are low in one category, you can often make up for it by having a larger number in another category.

Generally speaking your data will not arrive in a perfect form for Personalize, and will take some modification to be structured correctly. This notebook looks to guide you through all of that. 

To begin with, we are going to use the [Last.FM](https://grouplens.org/datasets/hetrec-2011/) dataset. These are records of the music listening behavior of its users. The data fits our guidelines with a large number for users, items, and interactions.

First, you will download the dataset and unzip it in a new folder using the code below.

In [1]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

--2020-05-26 19:25:56--  http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  9.60MB/s    in 0.3s    

2020-05-26 19:25:57 (9.60 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


Take a look at the data files you have downloaded.

In [2]:
!ls $data_dir

artists.dat		  tags.dat	    user_taggedartists.dat
hetrec2011-lastfm-2k.zip  user_artists.dat  user_taggedartists-timestamps.dat
readme.txt		  user_friends.dat


At present not much is known about the data other than we seem to have many .dat files and a README. Opening the README will tell us about the overall structure of this data. This is a step you probably can skip with custom data unless the data source is coming from an external team.

From the README, we can see that there are multiple interaction types in this dataset. Interactions between users marking each other as friends, interactions from users listening to artists, and interactions from tags assigned to users and artists.

In this case, we are focusing on the users, the artists, and the listening interactions. We have 1892 users, 17632 artists (our items in this case), and 92834 user-listened artist interactions. This is more than enough for us to get started with Personalize.

Continue reading through the README to get to the `Files` section. Most of the files in the dataset are not relevant to us, but the `users_artists.dat` file looks promising. The `Data format` section of the README provides more details on the contents of the file. This is where we encounter our first problem.

| userID | artistID | weight  |
|--------|----------|---------|
| 2      | 51       | 13883   |

Although there is interaction data between users and the artists they are listening to, these interactions are stored as weights instead of timestamps. We need user-item-timestamp interaction data for Amazon Personalize. 

If you take another look at the files in the dataset, you should see that `users_taggedartists-timestamps.dat` does contain timestamp data. So what if we use tagging behavior as our interaction data, instead of listening behavior? Can we assume that a user tagging an artist is an indication of positive sentiment? Normally, you would discuss with your customer, or someone who has domain knowledge, to understand if this interaction is suitable for the use case you want to solve. For now, we will assume that tagging behavior is suitable for our needs. 

The schema for the `user_taggedartists-timestamps.dat` is:

| userID | artistID | tagID | timestamp     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

If we remove the `tagID` attribute, we have exactly the format we need for Amazon Personalize.

## Prepare your data <a class="anchor" id="prepare"></a>
[Back to top](#top)

The next thing to be done is to load the data and confirm the data is in a good state, then save it to a CSV where it is ready to be used with Amazon Personalize.

To get started, import a collection of Python libraries commonly used in data science.

In [5]:
import time
from time import sleep
import json
from datetime import datetime

import boto3
import pandas as pd

Next,open the data file and take a look at the first several rows.

In [6]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

,userID artistID tagID timestamp
0,2\t52\t13\t1238536800000
1,2\t52\t15\t1238536800000
2,2\t52\t18\t1238536800000
3,2\t52\t21\t1238536800000
4,2\t52\t41\t1238536800000


Clearly the data did not load correctly. The default delimiter for CSV (comma-separated value) files is a comma (`,`), but in this case the file was saved with tab (`\t`) delimiters. So let's specify the correct delimiter and try loading the data again.

In [7]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
3,2,52,21,1238536800000
4,2,52,41,1238536800000


That's better. Now that the data has been successfully loaded into memory, let's extract some additional information. First, calculate some basic statistics from the data.

In [8]:
original_data.describe()

,userID,artistID,tagID,timestamp
count,186479.000000,186479.000000,186479.000000,1.864790e+05
mean,1035.600137,4375.845328,1439.582913,1.239204e+12
std,622.461272,4897.789595,2775.340279,4.299091e+10
min,2.000000,1.000000,1.000000,-4.287204e+11
25%,488.000000,686.000000,79.000000,1.209593e+12
50%,1021.000000,2203.000000,195.000000,1.243807e+12
75%,1624.000000,6714.000000,887.000000,1.275343e+12
max,2100.000000,18744.000000,12647.000000,1.304941e+12


This shows that we have a good range of values for `userID` and `artistID`. Next, it is always a good idea to confirm the data format.

In [9]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186479 entries, 0 to 186478
Data columns (total 4 columns):
userID       186479 non-null int64
artistID     186479 non-null int64
tagID        186479 non-null int64
timestamp    186479 non-null int64
dtypes: int64(4)
memory usage: 5.7 MB


From this, you can see that there are a total of 186,479 entries in the dataset, with 4 columns, and each cell stored as int64 format.

The int64 format is clearly suitable for `userID` and `artistID`. However, we need to diver deeper to understand the timestamps in the data. To use Amazon Personalize, you need to save timestamps in [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time) format.

Currently, the timestamp values are not human-readable. So let's grab an arbitrary timestamp value and figure out how to interpret it.

In [10]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000000


ValueError: year 41132 is out of range

Oops! For this particular timestamp value, the code rendered a year of 41,132. That's a bit far into the future for us, so clearly this was not the correct way to parse the data. We need a second attempt.

JavaScript records time in milliseconds and this is a collection of data from a web application, so let's divide the timestamp value by 1000 before applying our code.

In [11]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

2009-02-28 23:00:00


February, 2009 feels much more realistic for our dataset. We don't need human-readable timestamps to use Amazon Personalize, but we do want the dates to be realistic, so now move forward by transforming each timestamp in the dataset away from the JavaScript milliseconds format. 

In [12]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1.238537e+09
1,2,52,15,1.238537e+09
2,2,52,18,1.238537e+09
3,2,52,21,1.238537e+09
4,2,52,41,1.238537e+09


Do a quick sanity check on the transformed dataset by picking an arbitrary timestamp and transforming it to a human-readable format.

In [13]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000.0
2009-02-28 23:00:00


This date makes sense as a timestamp, so we can continue formatting the rest of the data. Remember, the data we need is user-item-interaction data, which is `userID`, `artistID`, and `timestamp` in this case. Our dataset has an additional column, `tagID`, which can be dropped from the dataset.

In [14]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

,userID,artistID,timestamp
0,2,52,1.238537e+09
1,2,52,1.238537e+09
2,2,52,1.238537e+09
3,2,52,1.238537e+09
4,2,52,1.238537e+09


After manipulating the data, always confirm if the data format has changed.

In [15]:
interactions_df.dtypes

userID         int64
artistID       int64
timestamp    float64
dtype: object

In this case, the timestamp column has changed from int64 to float64. So let's change the format back to int64.

In [16]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

userID       int64
artistID     int64
timestamp    int64
dtype: object

 Amazon Personalize has default column names for users, items, and timestamp. These default column names are `USER_ID`, `ITEM_ID`, AND `TIMESTAMP`. So the final modification to the dataset is to replace the existing column headers with the default headers.

In [17]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

That's it! At this point the data is ready to go, and we just need to save it as a CSV file.

In [18]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Create dataset groups and the interactions dataset <a class="anchor" id="group_dataset"></a>
[Back to top](#top)

The highest level of isolation and abstraction with Amazon Personalize is a *dataset group*. Information stored within one of these dataset groups has no impact on any other dataset group or models created from one - they are completely isolated. This allows you to run many experiments and is part of how we keep your models private and fully trained only on your data. 

Before importing the data prepared earlier, there needs to be a dataset group and a dataset added to it that handles the interactions.

Dataset groups can house the following types of information:

* User-item-interactions
* Event streams (real-time interactions)
* User metadata
* Item metadata

Before we create the dataset group and the dataset for our interaction data, let's validate that your environment can communicate successfully with Amazon Personalize.

In [19]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Create the dataset group

The following cell will create a new dataset group with the name `personalize-poc-lastfm`.

In [20]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-batch-recommendations-dg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/personalize-batch-recommendations-dg",
  "ResponseMetadata": {
    "RequestId": "ad7b4996-81a4-4896-8d69-29725f934988",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 19:27:56 GMT",
      "x-amzn-requestid": "ad7b4996-81a4-4896-8d69-29725f934988",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Before we can use the dataset group, it must be active. This can take a minute or two. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the dataset group every second, up to a maximum of 3 hours.

In [21]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Now that you have a dataset group, you can create a dataset for the interaction data.

### Create the dataset

First, define a schema to tell Amazon Personalize what type of dataset you are uploading. There are several reserved and mandatory keywords required in the schema, based on the type of dataset. More detailed information can be found in the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Here, you will create a schema for interactions data, which needs the `USER_ID`, `ITEM_ID`, and `TIMESTAMP` fields. These must be defined in the same order in the schema as they appear in the dataset.

In [22]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-batch-recommendations-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/personalize-batch-recommendations-interactions",
  "ResponseMetadata": {
    "RequestId": "7b3d0693-88e2-4323-b004-49852911c121",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 19:34:08 GMT",
      "x-amzn-requestid": "7b3d0693-88e2-4323-b004-49852911c121",
      "content-length": "112",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


With a schema created, you can create a dataset within the dataset group. Note, this does not load the data yet. This will happen a few steps later.

In [23]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-batch-recommendations-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/personalize-batch-recommendations-dg/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "bac3b30a-a086-4c56-b437-a8594fab4de4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 19:34:24 GMT",
      "x-amzn-requestid": "bac3b30a-a086-4c56-b437-a8594fab4de4",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Configure an S3 bucket and an IAM  role <a class="anchor" id="bucket_role"></a>
[Back to top](#top)

So far, we have downloaded, manipulated, and saved the data onto the Amazon EBS instance attached to instance running this Jupyter notebook. However, Amazon Personalize will need an S3 bucket to act as the source of your data, as well as IAM roles for accessing that bucket. Let's set all of that up.

Use the metadata stored on the instance underlying this Amazon SageMaker notebook, to determine the region it is operating in. If you are using a Jupyter notebook outside of Amazon SageMaker, simply define the region as a string below. The Amazon S3 bucket needs to be in the same region as the Amazon Personalize resources we have been creating so far.

In [24]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Amazon S3 bucket names are globally unique. To create a unique bucket name, the code below will append the string `personalizepoc` to your AWS account number. Then it creates a bucket with this name in the region discovered in the previous cell.

In [25]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-personalize-batch-recommendations"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

144386903708-personalize-batch-recommendations


### Upload data to S3

Now that your Amazon S3 bucket has been created, upload the CSV file of our user-item-interaction data. 

In [26]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### Set the S3 bucket policy
Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

In [27]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '265A8A8CB0820F2A',
  'HostId': 'otqfc/4JiKCplh1U5JWiZhSjIA7s7650EC+g7eFuJuCbsdkPPhVtFUWA9MTAc1r2j3jn/ncHVRc=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'otqfc/4JiKCplh1U5JWiZhSjIA7s7650EC+g7eFuJuCbsdkPPhVtFUWA9MTAc1r2j3jn/ncHVRc=',
   'x-amz-request-id': '265A8A8CB0820F2A',
   'date': 'Tue, 26 May 2020 19:34:40 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Create an IAM role

Amazon Personalize needs the ability to assume roles in AWS in order to have the permissions to execute certain tasks. Let's create an IAM role and attach the required policies to it. The code below attaches very permissive policies; please use more restrictive policies for any production application.

In [28]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleBatchRecommendations"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeRoleBatchRecommendations


## Import the interactions data <a class="anchor" id="import"></a>
[Back to top](#top)

Earlier you created the dataset group and dataset to house your information, so now you will execute an import job that will load the data from the S3 bucket into the Amazon Personalize dataset. 

In [29]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-batch-recommendations",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/personalize-batch-recommendations",
  "ResponseMetadata": {
    "RequestId": "cf1914f2-1a32-403d-be93-f0f4ffc6d5ea",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 19:36:34 GMT",
      "x-amzn-requestid": "cf1914f2-1a32-403d-be93-f0f4ffc6d5ea",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Before we can use the dataset, the import job must be active. Execute the cell below and wait for it to show the ACTIVE status. It checks the status of the import job every second, up to a maximum of 3 hours.

Importing the data can take some time, depending on the size of the dataset. In this workshop, the data import job should take around 15 minutes.

In [30]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 53.2 ms, sys: 26.9 ms, total: 80.2 ms
Wall time: 16min 1s


When the dataset import is active, you are ready to start building models with SIMS, Personalized-Ranking, Popularity-Count, and HRNN. This process will continue in other notebooks. Run the cell below before moving on to store a few values for usage in the next notebooks.

## Create solutions <a class="anchor" id="solutions"></a>
[Back to top](#top)

In this notebook, you will create solutions with the following recipe:

1. HRNN


In Amazon Personalize, a specific variation of an algorithm is called a recipe. Different recipes are suitable for different situations. A trained model is called a solution, and each solution can have many versions that relate to a given volume of data when the model was trained.

To start, we will list all the recipes that are supported. This will allow you to select one and use that to build your model.

In [31]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

The output is just a JSON representation of all of the algorithms mentioned in the introduction.

Next we will select specific recipes and build models with them.

### HRNN

HRNN (hierarchical recurrent neural network) is one of the more advanced recommendation models that you can use and it allows for real-time updates of recommendations based on user behavior. It also tends to outperform other approaches, like collaborative filtering. This recipe takes the longest to train, so let's start with this recipe first.

For our use case, using the LastFM data, we can use HRNN to recommend new artists to a user based on the user's previous artist tagging behavior. Remember, we used the tagging data to represent positive interactions between a user and an artist.

First, select the recipe by finding the ARN in the list of recipes above.

In [32]:
HRNN_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

#### Create the solution

First you create a solution using the recipe. Although you provide the dataset ARN in this step, the model is not yet trained. See this as an identifier instead of a trained model.

In [33]:
hrnn_create_solution_response = personalize.create_solution(
    name = "personalize-batch-recommendations-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = HRNN_recipe_arn
)

hrnn_solution_arn = hrnn_create_solution_response['solutionArn']
print(json.dumps(hrnn_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-batch-recommendations-hrnn",
  "ResponseMetadata": {
    "RequestId": "ac50ccf0-9fee-45ba-aad4-84d6ed6474cc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 19:53:27 GMT",
      "x-amzn-requestid": "ac50ccf0-9fee-45ba-aad4-84d6ed6474cc",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Create the solution version

Once you have a solution, you need to create a version in order to complete the model training. The training can take a while to complete, upwards of 25 minutes, and an average of 40 minutes for this recipe with our dataset. Normally, we would use a while loop to poll until the task is completed. However the task would block other cells from executing, and the goal here is to create many models and deploy them quickly. So we will set up the while loop for all of the solutions further down in the notebook. There, you will also find instructions for viewing the progress in the AWS console.

In [34]:
hrnn_create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

In [35]:
hrnn_solution_version_arn = hrnn_create_solution_version_response['solutionVersionArn']
print(json.dumps(hrnn_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-batch-recommendations-hrnn/8a2e378f",
  "ResponseMetadata": {
    "RequestId": "487d449a-6349-4183-8a14-ac1ca0fb4136",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 19:53:30 GMT",
      "x-amzn-requestid": "487d449a-6349-4183-8a14-ac1ca0fb4136",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### View solution creation status

As promised, how to view the status updates in the console:

* In another browser tab you should already have the AWS Console up from opening this notebook instance. 
* Switch to that tab and search at the top for the service `Personalize`, then go to that service page. 
* Click `View dataset groups`.
* Click the name of your dataset group, most likely something with POC in the name.
* Click `Solutions and recipes`.
* You will now see a list of all of the solutions you created above,  including a column with the status of the solution versions. Once it is `Active`, your solution is ready to be reviewed. It is also capable of being deployed.

Or simply run the cell below to keep track of the solution version creation status.

In [36]:
in_progress_solution_versions = [
    hrnn_solution_version_arn,
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solution build is still in progress
At least one solutio

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [47]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

,name,url,pictureURL
id,,,
1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


By defining the ID column as the index column it is trivial to return an artist by just querying the ID.

In [48]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Earth, Wind & Fire


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [49]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

Now let's test a few simple values to check our error catching.

In [50]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Earth, Wind & Fire
Error obtaining artist
Error obtaining artist


Great! Now we have a way of rendering results. 

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [38]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

,artistID,weight
userID,,
2,51,13883
2,52,11690
2,53,11351
2,54,10300
2,55,8983


In [39]:
# Get the user list
batch_users = users_df.sample(3).index.tolist()

# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [41]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "1382"}
{"userId": "1874"}
{"userId": "1788"}


Upload the file to S3 and save the path as a variable for later.

In [42]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://144386903708-personalize-batch-recommendations/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [43]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://144386903708-personalize-batch-recommendations/


Now just make the call to kick off the batch export process.

In [44]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = hrnn_solution_version_arn,
    jobName = "Batch-Inference-Job-HRNN",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 25 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [45]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  08:41:41 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: ACTIVE
Import Completed on:  09

Once the batch recommendations job has finished processing, we can grab the output uploaded to S3 and parse it.

In [51]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            artist = get_artist_by_id(item)
            recommendation_list.append(artist)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 1382,User: 1874,User: 1788
0,Britney Spears,Subheim,Miley Cyrus
1,Muse,Code 64,Christina Aguilera
2,Christina Aguilera,M.O.P.,Britney Spears
3,Lady Gaga,Neuromantik,Taylor Swift
4,The Beatles,Cassidy,Glee Cast
5,Depeche Mode,Santa Hates You,Shakira
6,citizengreen,A-Team,Jennifer Lopez
7,Eminem,Fantazja,Mariah Carey
8,Glee Cast,Snow in Mexico,Beyoncé
9,Girls Aloud,Pride and Fall,Katy Perry


## Clean up solutions

Next, clean up the solutions. The code below will list all of the solutions in your account.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

Look through the list of ARNs to determine which solution you want to delete. Then use the code below to delete the it by inserting the ARN.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## Clean up datasets

Next, clean up the datasets. The code below will list all of the datasets in your account.

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

Look through the list of ARNs to determine which dataset you want to delete. Then use the code below to delete the it by inserting the ARN.

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## Clean up the schemas

Next, clean up the schemas. The code below will list all of the schemas in your account.

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

Look through the list of ARNs to determine which schema you want to delete. Then use the code below to delete the it by inserting the ARN.

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## Clean up the dataset groups

Finally, clean up the dataset group. The code below will list all of the dataset groups in your account.

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

Look through the list of ARNs to determine which dataset group you want to delete. Then use the code below to delete the it by inserting the ARN.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## Clean up the S3 bucket and IAM role

Optionally, you can delete the IAM role and the S3 bucket which we created throughout the workshop.

Start by listing all of the IAM roles in your account with the code below.

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

Identify the name of the role you want to delete.

You cannot delete an IAM role which still has policies attached to it. So after you have identified the relevant role, let's list the attached policies of that role.

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

You need to detach the policies in the result above using the code below. Repeat for each attached policy.

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

Finally, you should be able to delete the IAM role.

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

To delete an S3 bucket, it first needs to be empty. The easiest way to delete an S3 bucket, is just to navigate to S3 in the AWS console, delete the objects in the bucket, and then delete the S3 bucket itself.